<a href="https://colab.research.google.com/github/AneesahAneesah/aneesah_INFO5731_Spring2021/blob/main/Project_First_Deliverable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob

import numpy as np
import pandas as pd
import sys

In [2]:
TWITTER_KEY = "Hh4zaN2rQgsR7IMWpQhIcFpEr"
TWITTER_SECRET_KEY = "VxuPnxN0QKP47s44Tn1807gPRRwxJXCwiJS3U6k16WiJFehdpn"
auth = tweepy.AppAuthHandler(TWITTER_KEY, TWITTER_SECRET_KEY)
api = tweepy.API(auth, wait_on_rate_limit=True,
				   wait_on_rate_limit_notify=True)

if (not api):
    print ("Can't Authenticate")
    sys.exit(-1)

In [12]:
searchQuery = '#txwx' 

maxTweets = 5000
Filter_Retweets = True

tweetsPerQry = 100  
tweet_lst = []

if Filter_Retweets:
  searchQuery = searchQuery + ' -filter:retweets'  

sinceId = None

max_id = -10000000000

tweetCount = 0
print("Downloading max {0} tweets".format(maxTweets))
while tweetCount < maxTweets:
    try:
        if (max_id <= 0):
            if (not sinceId):
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry, lang="en")
            else:
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                        lang="en", since_id=sinceId)
        else:
            if (not sinceId):
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                        lang="en", max_id=str(max_id - 1))
            else:
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                        lang="en", max_id=str(max_id - 1),
                                        since_id=sinceId)
        if not new_tweets:
            print("No more tweets found")
            break
        for tweet in new_tweets:
          if hasattr(tweet, 'reply_count'):
            reply_count = tweet.reply_count
          else:
            reply_count = 0
          if hasattr(tweet, 'retweeted'):
            retweeted = tweet.retweeted
          else:
            retweeted = "NA"
            
         
          topic = searchQuery[:searchQuery.find('-')].capitalize().strip()
          
          tweetDate = tweet.created_at.date()
          
          tweet_lst.append([tweetDate, topic, 
                      tweet.id, tweet.user.screen_name, tweet.user.name, tweet.text, tweet.favorite_count, 
                      reply_count, tweet.retweet_count, retweeted])

        tweetCount += len(new_tweets)
        print("Downloaded {0} tweets".format(tweetCount))
        max_id = new_tweets[-1].id
    except tweepy.TweepError as e:
        
        print("some error : " + str(e))
        break


print("Downloaded {0} tweets".format(tweetCount))

Downloaded 100 tweets
Downloaded 200 tweets
Downloaded 300 tweets
Downloaded 400 tweets
Downloaded 500 tweets
Downloaded 600 tweets
Downloaded 700 tweets
Downloaded 800 tweets
Downloaded 900 tweets
Downloaded 1000 tweets
Downloaded 1100 tweets
Downloaded 1200 tweets
Downloaded 1300 tweets
Downloaded 1400 tweets
Downloaded 1500 tweets
Downloaded 1600 tweets
Downloaded 1700 tweets
Downloaded 1800 tweets
Downloaded 1900 tweets
Downloaded 2000 tweets
Downloaded 2100 tweets
Downloaded 2200 tweets
Downloaded 2300 tweets
Downloaded 2400 tweets
Downloaded 2500 tweets
Downloaded 2600 tweets
Downloaded 2700 tweets
Downloaded 2800 tweets
Downloaded 2900 tweets
Downloaded 3000 tweets
Downloaded 3100 tweets
Downloaded 3200 tweets
Downloaded 3300 tweets
Downloaded 3400 tweets
Downloaded 3500 tweets
Downloaded 3600 tweets
Downloaded 3700 tweets
Downloaded 3800 tweets
Downloaded 3900 tweets
Downloaded 4000 tweets
Downloaded 4100 tweets
Downloaded 4200 tweets
Downloaded 4300 tweets
Downloaded 4400 twee

In [13]:
# load it into a pandas dataframe
tweet_df1 = pd.DataFrame(tweet_lst, columns=['tweet_dt', 'topic', 'id', 'username', 'name', 'tweet', 'like_count', 'reply_count', 'retweet_count', 'retweeted'])
tweet_df1.to_csv('tweets1.csv')
tweet_df1

,tweet_dt,topic,id,username,name,tweet,like_count,reply_count,retweet_count,retweeted
0,2021-03-27,#txwx,1375643753365565442,albertquiles,J. A. Q.,#Weather #LongviexTX #KTXLONGV13 10:00 PM Temp...,0,0,0,False
1,2021-03-27,#txwx,1375643751239008264,sachseweather,SachseWeather,10:00 PM Weather Conditions at Sachse #dfwtx #...,0,0,0,False
2,2021-03-27,#txwx,1375643750001737728,lahwx,Lake Alan Henry Weather,Lake Alan Henry - Justiceburg #TX @ 10:00 PM -...,0,0,0,False
3,2021-03-27,#txwx,1375640562292961283,KXIITV,KXII News 12,"Saturday evening, after a day with temps in th...",1,0,1,False
4,2021-03-27,#txwx,1375639174267736072,CBS4WarnWeather,CBS4Warn Weather,A few spotty rain areas possible tomorrow with...,1,0,0,False
...,...,...,...,...,...,...,...,...,...,...
4908,2021-03-19,#txwx,1372851094473355264,ARomero_WX,Alberto Romero,Good morning! The time now is 5:00 AM! Here is...,3,0,0,False
4909,2021-03-19,#txwx,1372850337615388674,lahwx,Lake Alan Henry Weather,Lake Alan Henry - Justiceburg #TX @ 05:00 AM -...,0,0,1,False
4910,2021-03-19,#txwx,1372850336369639425,albertquiles,J. A. Q.,#Weather #LongviexTX #KTXLONGV13 05:00 AM Temp...,0,0,0,False
4911,2021-03-19,#txwx,1372848498920497154,BrendenMincheff,Brenden Mincheff,"Gooood morning, West Texas! ☀️ It's the last d...",2,0,0,False


In [14]:
import nltk
nltk.download()


NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> all
Command 'all' unrecognized

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all
       | 
       | Downloading package abc to /root/nltk_data...
       |   Package abc is already up-to-date!
       | Downloading package alpino to /root/nltk_data...
       |   Package alpino is already up-to-date!
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Package biocreative_ppi is already up-to-date!
       | Downloading package brown to /root/nltk_data...
       |   Package

True

In [15]:
#removing noise i.e special characters and punctuation

import re
tweet_df1['Reviews after removing noise']=tweet_df1['tweet'].str.replace('[^\w\s]','')
tweet_df1['Reviews after removing noise'] = tweet_df1['Reviews after removing noise'].apply(lambda x: ''.join(re.sub(r"[^a-zA-Z0-9]+", ' ', character) for character in x ))

In [16]:
#removing digits

tweet_df1['Reviews after removing digits'] = tweet_df1['Reviews after removing noise'].apply(lambda x: ''.join([i for i in x if not i.isdigit()]))

In [17]:
#removing stopwords
#importing stopwords collection from nltk
#already existing stopwords

from nltk.corpus import stopwords
stop = stopwords.words('english')
tweet_df1['Reviews after removing StopWords'] = tweet_df1['Reviews after removing digits'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [18]:
#lowercase all text
tweet_df1['Reviews after converting into LowerCase'] = tweet_df1['Reviews after removing StopWords'].apply(lambda x: " ".join(x.lower() for x in x.split()))

In [19]:
tweet_df1

,tweet_dt,topic,id,username,name,tweet,like_count,reply_count,retweet_count,retweeted,Reviews after removing noise,Reviews after removing digits,Reviews after removing StopWords,Reviews after converting into LowerCase
0,2021-03-27,#txwx,1375643753365565442,albertquiles,J. A. Q.,#Weather #LongviexTX #KTXLONGV13 10:00 PM Temp...,0,0,0,False,Weather LongviexTX KTXLONGV13 1000 PM Temp 640...,Weather LongviexTX KTXLONGV PM Temp F Wind mp...,Weather LongviexTX KTXLONGV PM Temp F Wind mph...,weather longviextx ktxlongv pm temp f wind mph...
1,2021-03-27,#txwx,1375643751239008264,sachseweather,SachseWeather,10:00 PM Weather Conditions at Sachse #dfwtx #...,0,0,0,False,1000 PM Weather Conditions at Sachse dfwtx txw...,PM Weather Conditions at Sachse dfwtx txwx sa...,PM Weather Conditions Sachse dfwtx txwx sachse...,pm weather conditions sachse dfwtx txwx sachse...
2,2021-03-27,#txwx,1375643750001737728,lahwx,Lake Alan Henry Weather,Lake Alan Henry - Justiceburg #TX @ 10:00 PM -...,0,0,0,False,Lake Alan Henry Justiceburg TX 1000 PM Temp...,Lake Alan Henry Justiceburg TX PM Temp HL...,Lake Alan Henry Justiceburg TX PM Temp HL Hum ...,lake alan henry justiceburg tx pm temp hl hum ...
3,2021-03-27,#txwx,1375640562292961283,KXIITV,KXII News 12,"Saturday evening, after a day with temps in th...",1,0,1,False,Saturday evening after a day with temps in the...,Saturday evening after a day with temps in the...,Saturday evening day temps upper pushing retur...,saturday evening day temps upper pushing retur...
4,2021-03-27,#txwx,1375639174267736072,CBS4WarnWeather,CBS4Warn Weather,A few spotty rain areas possible tomorrow with...,1,0,0,False,A few spotty rain areas possible tomorrow with...,A few spotty rain areas possible tomorrow with...,A spotty rain areas possible tomorrow cooler b...,a spotty rain areas possible tomorrow cooler b...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4908,2021-03-19,#txwx,1372851094473355264,ARomero_WX,Alberto Romero,Good morning! The time now is 5:00 AM! Here is...,3,0,0,False,Good morning The time now is 500 AM Here is YO...,Good morning The time now is AM Here is YOUR ...,Good morning The time AM Here YOUR outdoor for...,good morning the time am here your outdoor for...
4909,2021-03-19,#txwx,1372850337615388674,lahwx,Lake Alan Henry Weather,Lake Alan Henry - Justiceburg #TX @ 05:00 AM -...,0,0,1,False,Lake Alan Henry Justiceburg TX 0500 AM Temp...,Lake Alan Henry Justiceburg TX AM Temp HL...,Lake Alan Henry Justiceburg TX AM Temp HL Hum ...,lake alan henry justiceburg tx am temp hl hum ...
4910,2021-03-19,#txwx,1372850336369639425,albertquiles,J. A. Q.,#Weather #LongviexTX #KTXLONGV13 05:00 AM Temp...,0,0,0,False,Weather LongviexTX KTXLONGV13 0500 AM Temp 464...,Weather LongviexTX KTXLONGV AM Temp F Wind mp...,Weather LongviexTX KTXLONGV AM Temp F Wind mph...,weather longviextx ktxlongv am temp f wind mph...
4911,2021-03-19,#txwx,1372848498920497154,BrendenMincheff,Brenden Mincheff,"Gooood morning, West Texas! ☀️ It's the last d...",2,0,0,False,Gooood morning West Texas Its the last day of...,Gooood morning West Texas Its the last day of...,Gooood morning West Texas Its last day workwee...,gooood morning west texas its last day workwee...
